### Exploração dos dados

In [1]:
import pandas as pd
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns

c:\Users\sensix\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 1º Passo: Abrir e intepretar arquivos

In [2]:
# Ofertas
"""
id (string) - id da oferta.
● offertype (string) - o tipo da oferta. i.e. BOGO, discount, informational.
● minvalue (int) - o valor mínimo que precisa ser gasto para que a oferta seja ativada.
● discountvalue (int) - o valor do desconto a ser aplicado, caso a oferta seja ativada.
"""
df_offers = pd.read_json(r"\data\offers.json")

# Clientes
"""
● age (int) - idade do cliente no momento da criação da conta.
● registeredon (int) - a data em que o cliente criou a conta.
● gender (string) - gênero do cliente (algumas entradas podem conter '0' para outras
opções além de M ou F).
● id (string) - id do cliente.
● creditcardlimit (float) - o limite do cartão de crédito do cliente registrado no momento
da criação da conta.
"""
df_clients = pd.read_json(r"\data\clients.json")

# Transações
"""
● event (str) - descrição do evento (transação, oferta recebida, oferta visualizada,
oferta concluída, etc).
● accountid (str) - id do cliente.
● timesinceteststart (int) - tempo passada desde o começo do teste, em dias. Os
dados começam em t=0.
● value - (json) - pode registra o offerid de oferta, o desconto concedido (key reward)
ou o valor da transação, dependendo do tipo de evento
"""
df_transactions = pd.read_json(r"\data\transactions.json")

### 2° Passo indentificar outliers


#### 2.1 Tratando dados de Ofertas

In [3]:
# Neste dataset,valores separar a coluna channels em colunas especificas para cada canal de envio do oferta de cupom 
df_offers['channels'] = df_offers['channels'].astype(str)
df_offers['channels'] = df_offers['channels'].str.replace('[', '').str.replace(']', '')
df_channels = df_offers['channels'].str.get_dummies(sep=', ')

# Substituir valores 1 por 'yes' e 0 por ''
df_channels = df_channels.replace({1: 'yes', 0: ''})

# Concatenar o dataframe original com o novo dataframe de canais
df_offers = pd.concat([df_offers, df_channels], axis=1)
df_offers.drop('channels', axis=1, inplace=True)

#### 2.2 Tratando dados de Clientes

In [4]:
# Primeiro vamos transformar a coluna registeredon em datetime
df_clients['date'] = pd.to_datetime(df_clients['registered_on'], format='%Y%m%d')
df_clients.drop('registered_on', axis=1, inplace=True)
df_clients

# Verificar se a tem id repetidos
if df_clients['id'].duplicated().sum() == 0:
    print("Não há id repetidos")

# Analisando os dados do dataset de clientes, identificamos alguns outliers na coluna age e gender, 
# onde temos idades muito elevadas e gêneros diferentes de M e F.
# Curiosamente para esses caso temos valores de creditcardlimit vazios.

# Remover valores de None na coluna gender
df_clients = df_clients[df_clients['gender'].isin(['M', 'F'])]

# Remover outliers na coluna age
df_clients = df_clients[df_clients['age'] < 100]
df_clients

# Verificar se a coluna credit_card_limit tem valores NaN
if df_clients['credit_card_limit'].isna().sum() == 0:
    print("Dataset limpo")
df_clients

Não há id repetidos
Dataset limpo


,age,gender,id,credit_card_limit,date
1,55,F,0610b486422d4921ae7d2bf64640c50b,112000.0,2017-07-15
3,75,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0,2017-05-09
5,68,M,e2127556f4f64592b11af22de27a7932,70000.0,2018-04-26
8,65,M,389bc3fa690240e798340f5a15918d5c,53000.0,2018-02-09
12,58,M,2eeac8d8feae4a8cad5a6af0499a211d,51000.0,2017-11-11
...,...,...,...,...,...
16995,45,F,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,2018-06-04
16996,61,M,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,2018-07-13
16997,49,M,01d26f638c274aa0b965d24cefe3183f,73000.0,2017-01-26
16998,83,F,9dc1421481194dcd9400aec7c9ae6366,50000.0,2016-03-07


#### 2.3 Tratando dados das transações

In [5]:
# Analisandos os dados de value, identificamos que a coluna 'value' é um json, com valores que podem ser float ou strings.
df_transactions['value'] = df_transactions['value'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df_values = json_normalize(df_transactions['value'])
df_transactions = pd.concat([df_transactions, df_values], axis=1)
df_transactions.drop('value', axis=1, inplace=True)

# Verificar inconsistências entre 'offerid' e 'offer_id', para indenticar se temos valores repetidos
df_transactions['offer_id'] = df_transactions['offer id'].apply(lambda x: '' if pd.isna(x) else x)
inconsistencies = df_transactions[(df_transactions['offer id'].notna()) & (df_transactions['offer_id'] == '')]

print("Inconsistências encontradas:")
print(inconsistencies)

# Não a Inconsistências entre as colunas vamos juntar as duas colunas em uma só
df_transactions['offer_id'] = df_transactions['offer id'].combine_first(df_transactions['offer_id'])
df_transactions.drop('offer id', axis=1, inplace=True)
df_transactions


Inconsistências encontradas:
Empty DataFrame
Columns: [event, account_id, time_since_test_start, amount, offer id, offer_id, reward]
Index: []


,event,account_id,time_since_test_start,amount,offer_id,reward
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0.00,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,offer received,a03223e636434f42ac4c3df47e8bac43,0.00,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,offer received,e2127556f4f64592b11af22de27a7932,0.00,NaN,2906b810c7d4411798c6938adc9daaa5,NaN
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0.00,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,offer received,68617ca6246f4fbc85e91a2a49552598,0.00,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN
...,...,...,...,...,...,...
306529,transaction,b3a1272bc9904337b331bf348c3e8c17,29.75,1.59,,NaN
306530,transaction,68213b08d99a4ae1b0dcb72aebd9aa35,29.75,9.53,,NaN
306531,transaction,a00058cf10334a308c68e7631c529907,29.75,3.61,,NaN
306532,transaction,76ddbd6576844afe811f1a3c0fbb5bec,29.75,3.53,,NaN


### 3° Passo Juntar e relacionar os datasets

In [6]:
# Relacionar as ofertas aos clientes
df_transactions_clients = pd.merge(df_transactions, df_clients, left_on='account_id', right_on='id', how='left')
df_transactions_clients.drop('id', axis=1, inplace=True)

# Relacinar as transações com as ofertas
df_transactions_clients_offers = pd.merge(df_transactions_clients, df_offers, left_on='offer_id', right_on='id', how='left')
df_transactions_clients_offers.drop('id', axis=1, inplace=True)
df_transactions_clients_offers.rename(columns={'event': 'evento', 
                                               'account_id': 'id_cliente', 
                                               'time_since_test_start': 'time',
                                               'amount':'quantia',
                                               'offer_id':'id_oferta',
                                               'reward':'cupom',
                                               'age':'idade',
                                               'gender':'genero',
                                               'credit_card_limit':'limite_cartao',
                                               'date':'data',
                                               'min_value':'valor_minimo_compra',
                                               'duration':'duracao',
                                               'offer_type':'tipo_oferta',
                                               'discount_value':'valor_desconto'
                                               }, inplace=True)

### 4° Passo Salvar dataset

In [7]:
df_transactions_clients_offers.to_csv(r"\data\df_transactions_clients_offers.csv", index=False)